In [16]:
import pandas as pd
import requests

In [17]:
def get_relevent_lists(dataset: str):
    df = pd.read_csv(f'./search/datasets/{dataset}/{dataset}_qrels.csv')
    queries = df['query_id'].tolist()
    docs = df['doc_id'].tolist()
    relevance = df['relevance'].tolist()
    relevent_lists = {}
    for i in range(0, len(relevance)):
        if relevance[i] != 0:
            if queries[i] not in relevent_lists:
                relevent_lists[queries[i]] = []
            relevent_lists[queries[i]].append(docs[i])
    return relevent_lists

In [18]:
def get_retrieved_lists(dataset: str):
    df = pd.read_csv(f'./search/datasets/{dataset}/{dataset}_queries.csv')
    query_id = df['query_id'].tolist()
    text = df['text'].tolist()
    retrieved_lists = {}
    for i in range(0, len(query_id)):
        response = requests.get(f"http://127.0.0.1:8000/search/{dataset}", params={"query": text[i]})
        results = response.json()
        retrieved_lists[query_id[i]] = results["ID"]
    return retrieved_lists

In [19]:
def precision_at_10(retrieved_list, relevant_list):
    top_10_retrieved = retrieved_list[:10]
    num_relevant = sum([1 for doc in top_10_retrieved if doc in relevant_list])
    p_at_10 = num_relevant / 10.0
    return p_at_10

In [20]:
def recall_at_10(retrieved_list, relevant_list):
    top_10_retrieved = retrieved_list[:10]
    num_relevant_in_top_10 = sum([1 for doc in top_10_retrieved if doc in relevant_list])
    r_at_10 = num_relevant_in_top_10 / len(relevant_list) if relevant_list else 0
    return r_at_10
 

In [21]:
def calculate_map_at_k(retrieved_lists, relevant_lists, k=10):
    ap_at_k = []
    for query, retrieved_list in retrieved_lists.items():
        if query in relevant_lists:
            relevant_list = set(relevant_lists[query])
            ap = 0
            correct = 0
            for i, doc in enumerate(retrieved_list[:k]):
                if doc in relevant_list:
                    correct += 1
                    ap += correct / (i + 1)
            if correct > 0:
                ap /= min(len(relevant_list), k)
                ap_at_k.append(ap)
    return sum(ap_at_k) / len(ap_at_k) if ap_at_k else 0

In [22]:
def calculate_mrr_at_k(retrieved_lists, relevant_lists, k=10):
    rr_at_k = []
    for query, retrieved_list in retrieved_lists.items():
        if query in relevant_lists:
            relevant_list = set(relevant_lists[query])
            for i, doc in enumerate(retrieved_list[:k]):
                if doc in relevant_list:
                    rr_at_k.append(1 / (i + 1))
                    break
    return sum(rr_at_k) / len(rr_at_k) if rr_at_k else 0


In [35]:
def evaluation(dataset: str):
    df = pd.read_csv(f'./search/datasets/{dataset}/{dataset}_queries.csv')
    query_id = df['query_id'].tolist()


    relevent_lists = get_relevent_lists(dataset)
    retrieved_lists = get_retrieved_lists(dataset)


    print(f"MAP for {dataset}:")
    print(calculate_map_at_k(retrieved_lists,relevent_lists,10))

    print(f"MRR for {dataset}:")
    print(calculate_mrr_at_k(retrieved_lists,relevent_lists,10))


    print(f"precision for the first five documents:")
    for id in query_id[0:5]:
        print(precision_at_10(retrieved_lists[id], relevent_lists[id]))


    print(f"recall for the first five documents:")
    for id in query_id[0:5]:
        print(recall_at_10(retrieved_lists[id], relevent_lists[id]))
    

In [36]:
evaluation("sara")

MAP for sara:
0.19198930430347397
MRR for sara:
0.6481223383662411
precision for the first five documents:
0.2
0.1
0.1
0.3
0.3
recall for the first five documents:
0.11764705882352941
0.058823529411764705
0.058823529411764705
0.2727272727272727
0.2727272727272727


In [37]:
evaluation("wikiren1ktraining")

MAP for wikiren1ktraining:
0.18207468705704702
MRR for wikiren1ktraining:
0.7154033041788154
precision for the first five documents:
0.6
0.1
0.1
0.1
0.1
recall for the first five documents:
1.0
0.16666666666666666
0.16666666666666666
0.1111111111111111
0.07142857142857142


In [ ]:
evaluation("antiquetrain")